# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](https://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section which can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [6]:
import numpy as np
import pandas as pd
import timeit

In [167]:
col_names = ['Sequence'] + list(np.arange(135, 1401))
ms_df = pd.DataFrame(columns = col_names)

In [168]:
ms_df

,Sequence,135,136,137,138,139,140,141,142,143,...,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400


In [169]:
def parse_name_line(file_line):
    parsed_name = file_line[6:]
    parsed_seq = file_line[6:].split('/')[0]
    parsed_ce = float(file_line[6:].split('/')[1].split('_')[-1][:-2])
    return parsed_name, parsed_seq, parsed_ce
    

In [170]:
def consider_conditions(seq, ce, max_seq_len, max_ce, min_ce):
    condition = True
    if len(seq) > max_seq_len:
        condition = False
    if max_ce <= ce or ce <= min_ce:
        condition = False
    return condition

In [171]:
def parse_peak_data(line):
    parsed_line = line.split('\t')
    peak_pos = round(float(parsed_line[0]))
    peak_intensity = float(parsed_line[1])
    return peak_pos, peak_intensity

In [172]:
max_seq_len=30
min_ce=36
max_ce=40
mz_min=135
mz_max=1400

with open('..\..\data\cptac2_mouse_hcd_selected.msp', 'r') as file:

    for line in file:
        
        line = line.strip()

        if line[:5] == 'Name:':
            conditions = False
            name, seq, ce = parse_name_line(line)
            conditions = consider_conditions(seq, ce, max_seq_len, max_ce, min_ce)

            if conditions == True:
                print(name)
                new_df_line = pd.Series([seq, *list(np.zeros(1266))], index = col_names, name=name)
                ms_df = ms_df.append(new_df_line)

        if conditions == True:
            if line[0:1] in np.arange(10).astype('str'): # line with peaks begins with a number
                position, intensity = parse_peak_data(line)

                if mz_min < position < mz_max:
                    if ms_df.loc[name, position] < intensity:
                        ms_df.at[name, position] = intensity

AAAFEDQENETVVVK/2_0_38.7eV
AAGTIQTSVQEVNSK/2_0_37.3eV
AAGVSVEPFWPGLFAK/2_0_39.3eV
AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)_37.6eV
AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)_39.1eV
AALLAELASLEADALR/2_0_38.1eV
AAPLSLCALTAVDQSVLLLKPEAK/3_1(6,C,CAM)_37eV
AAPQLPMEELVSLSK/2_0_37.8eV
AASGFNATEDAQTLR/2_0_37.7eV
AAVFNHFISDGVKK/2_0_37.3eV
ACNLEVILGFDGSR/2_1(1,C,CAM)_36.3eV
ADDDYDEPTDSLDAR/2_0_39.7eV
ADGLAILGVLMK/2_0_37eV
ADINEENEETITK/2_0_36.6eV
ADLPTGATTSAPVMLR/2_0_37.5eV
ADLPTGATTSAPVMLR/2_1(13,M,Oxidation)_37.9eV
ADMVIEAVFEDLGVK/2_0_39.8eV
AEANPPACYGTVLAEFQPLVEEPK/3_1(7,C,CAM)_38.8eV
AEDGAAPSPSSETPKK/2_0_36.8eV
AEEEQQGTGPGAGTAASK/2_0_39.5eV
AEEYEFLTPMEEAPK/2_0_37eV
AEEYEFLTPMEEAPK/2_1(9,M,Oxidation)_37eV
AEGIPEFYDYDVALVK/2_0_38eV
AELEHIAFNPSLVYLMDDFR/3_0_36.5eV
AELIQVTAGDPATLEYTVSGTPELKPK/3_0_39.3eV
AEMLQALASLEAVLLQTVYNTK/3_1(2,M,Oxidation)_37.1eV
AENACVPPFTVEVK/2_1(4,C,CAM)_36.5eV
AENPTNPGPGGSGYWR/2_0_38.9eV
AENYDISPADR/2_0_39eV
AEQGSGPVSGEKDVVFLIDGSEGVR/3_0_38.8eV
AETFTFHSDICTLPEK

--------

In [182]:
mz_min=135
mz_max=1400

col = ["Name", "Sequence"] + list(np.arange(mz_min,mz_max+1,1))
ms_df = pd.DataFrame(columns = col)
ms_df
def to_be_considered(line):
    conditions = True
    ce = float(line.split("_")[-1][:-2])
    if ce < min_ce or ce > max_ce:
        conditions = False
    elif len(sequence) > max_seq_len:
        conditions = False
    return conditions

def get_description_parameters(line):
    name = str(line[6:])
    sequence = line.split("/")[0]
    condition = to_be_considered(line)
    return name, sequence, condition
max_seq_len=30
min_ce=36
max_ce=40


In [183]:
max_seq_len=30
min_ce=36
max_ce=40


counter = 0
with open ('..\..\data\cptac2_mouse_hcd_selected.msp', 'r') as input_file:
        for line in input_file:
            line = line.strip()
            if line[:4] == "Name":
                name, sequence, condition = get_description_parameters(line)
                if condition == True:
                    ms_df = ms_df.append(pd.Series(0, index=ms_df.columns), ignore_index=True)
                    ms_df.at[counter, "Name"] = name
                    ms_df.at[counter, "Sequence"] = sequence
                    counter += 1

            if condition == True:
                if line[0:1] in ["0","1","2","3","4","5","6","7","8","9"]:
                    m_z = round(float(line.split("\t")[0]),0)
                    intensity = float(line.split("\t")[1])
                    if mz_min <= m_z <= mz_max:
                        empyt_column = False
                        if ms_df.at[counter - 1, m_z] != 0:
                            if ms_df.at[counter - 1, m_z] < intensity:
                                ms_df.at[counter - 1, m_z] = intensity
                        else:
                            ms_df.at[counter - 1, m_z] = intensity

-------

In [166]:
ms_df = ms_df.loc[(ms_df != 0).any(axis=1)]

In [ ]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """
    df = None
    seqs = None
    
    return df, seqs